In [2]:
import pandas as pd

df = pd.read_excel('../2_Preprocess/filtered_data.xlsx')
print(len(df))
df.sample(10)
# !pip install JPype1-1.1.2-cp36-cp36m-win_amd64.whl
# !pip install konlpy

153719


,text,class
128222,여성혐오란 남성과 여성을 동등한 주체로 보지않을떄 어쨰 남자도 결혼 잘해야 인생...,0
88772,"""중국의 역사는 몽골 원나라에 지배당한 이후로 끝났잖아? ㅋㅋ""",7
32665,초짜는 무슨 초짜 그동안 안걸렸을뿐 초짜염색하다 왜했을까 ㅋ 캭퉤퉤,0
98783,기자가 너무 엮네. 그럼 음주뺑소니 포함 음주운전 2번한 김흥국은?,0
146856,죄송합니다 문정부는 할줄아는게 없어요 ㅜㅜ어떻게 하면되죠? 라고 묻지도 않아요 그냥...,5
80893,"""중 국 놈 들 ..다 죽 어 버 려 라 !! 지구상에서 제일 쓰레기 인간들...""",7
100411,"""불교 기독교 없애라...결국 종교는 돈벌이 수단이다""",3
153132,하다하다. 자율주행차도 젠드? 성인지감수성 있는 자율주행차가 있어야 겠다. 김여사 ...,1
113468,품위있는 그녀다,0
152826,2:15 아직도 지능 떨어지는 몇몇들은 요가의 나라니 여자 혼자다녀도 괜찮다느니 위...,2


In [3]:
text_list = []

for idx in range(len(df)):
    tmp_text = df.loc[idx].text.replace("\n"," ")
    text_list.append(tmp_text)

text_to_save = '\n'.join(text_list)

with open('before_tokenized.txt', 'w',encoding='utf-8') as file:
    file.write(text_to_save)

In [4]:
import sys, re, argparse
# from khaiii import KhaiiiApi
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma


def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    elif tokenizer_name == "khaiii":
        tokenizer = KhaiiiApi()
    else:
        tokenizer = Mecab()
    return tokenizer


def tokenize(tokenizer_name, corpus_fname, output_fname, pos=False):
    tokenizer = get_tokenizer(tokenizer_name)
    with open(corpus_fname, 'r', encoding='utf-8') as f1, \
            open(output_fname, 'w', encoding='utf-8') as f2:
        for line in f1:
            sentence = line.replace('\n', '').strip()
            if tokenizer_name == "khaiii":
                tokens = []
                for word in tokenizer.analyze(sentence):
                    if pos:
                        tokens.extend([str(m) for m in word.morphs])
                    else:
                        tokens.extend([str(m).split("/")[0] for m in word.morphs])
            else:
                if pos:
                    tokens = tokenizer.pos(sentence)
                    tokens = [morph + "/" + tag for morph, tag in tokens]
                else:
                    tokens = tokenizer.morphs(sentence)
            tokenized_sent = ' '.join(post_processing(tokens))
            f2.writelines(tokenized_sent + '\n')


def post_processing(tokens):
    results = []
    for token in tokens:
        # 숫자에 공백을 주어서 띄우기
        processed_token = [el for el in re.sub(r"(\d)", r" \1 ", token).split(" ") if len(el) > 0]
        results.extend(processed_token)
    return results

In [5]:
tokenize('okt','./before_tokenized.txt','./okt_tokenized_text.txt')

In [6]:
after_listed = []
with open('./okt_tokenized_text.txt', 'r',encoding='utf-8') as file:
    for line in file:
        tmp_list = line.split(' ')
        tmp_list[-1] = tmp_list[-1].rstrip('\n')
        after_listed.append(tmp_list)
len(after_listed)

153719

In [7]:
import pickle
with open("./list_sentences.pkl","wb") as f:
    pickle.dump(after_listed, f)